In [1]:
import pandas as pd
import numpy as np
from score import score
from rec_utils import split_the_data
import warnings
warnings.filterwarnings('ignore')
from surprise import SVDpp, BaselineOnly
from surprise.model_selection import train_test_split as sur_tts
from surprise import Dataset, Reader, accuracy

import torch
from torch import nn
from torch import optim

In [2]:
user_data = pd.read_csv("../Course_Scraper/assets/augumented_data/augmented_user_rating.csv")

In [3]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_data, reader)
train, test = sur_tts(data, test_size=0.2, shuffle=True)

bsl_options = {
    "n_epochs": 100,
    "method": "sgd",
    "learning_rate": 0.001,
}
algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(train)
predictions = algo.test(test)
accuracy.rmse(predictions, verbose=True)

Estimating biases using sgd...
RMSE: 1.6404


1.6403549357897722

In [12]:
data, test = split_the_data(user_data)
train, val = split_the_data(data)

X_train = train.drop('rating', axis=1)
X_val = val.drop('rating', axis=1)

y_train = train['rating']
y_val = val['rating']

In [52]:
class RecSys(nn.Module):
    def __init__(self, input_: int = 2, output: int = 1):
        super(RecSys, self).__init__()
        self.fc1 = nn.Linear(2, 128)
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)
        # x = self.softmax(x)
        return x

In [53]:
model = RecSys()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)

In [54]:
for epoch in range(100):
    # convert the data to tensors
    tensor_Xtr = torch.tensor(X_train.to_numpy(), dtype=torch.float)
    tensor_Xva = torch.tensor(X_val.to_numpy(), dtype=torch.float)
    tensor_ytr = torch.tensor(y_train.to_numpy(), dtype=torch.float)
    tensor_yva = torch.tensor(y_val.to_numpy(), dtype=torch.float)

    model.train()
    outputs = model(tensor_Xtr)
    loss = torch.sqrt(criterion(outputs, tensor_ytr))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()

    with torch.no_grad():
        val_output = model(tensor_Xva)
        val_loss = torch.sqrt(criterion(val_output, tensor_yva))
        
    print(f'Epoch {epoch+1} - Train Loss: {loss.item()} - Val Loss {val_loss.item()}')

Epoch 1 - Train Loss: 4.316611289978027 - Val Loss 5.055158615112305
Epoch 2 - Train Loss: 4.251636028289795 - Val Loss 4.370450496673584
Epoch 3 - Train Loss: 4.203991413116455 - Val Loss 4.071814060211182
Epoch 4 - Train Loss: 4.167439937591553 - Val Loss 3.916330099105835
Epoch 5 - Train Loss: 4.131375789642334 - Val Loss 3.8279974460601807
Epoch 6 - Train Loss: 4.097622394561768 - Val Loss 3.77929949760437
Epoch 7 - Train Loss: 4.063751220703125 - Val Loss 3.747234582901001
Epoch 8 - Train Loss: 4.028741359710693 - Val Loss 3.720527172088623
Epoch 9 - Train Loss: 3.993804454803467 - Val Loss 3.6946182250976562
Epoch 10 - Train Loss: 3.960301637649536 - Val Loss 3.6663002967834473
Epoch 11 - Train Loss: 3.928234577178955 - Val Loss 3.6396963596343994
Epoch 12 - Train Loss: 3.8985655307769775 - Val Loss 3.614410161972046
Epoch 13 - Train Loss: 3.8681070804595947 - Val Loss 3.5876803398132324
Epoch 14 - Train Loss: 3.8383662700653076 - Val Loss 3.558375120162964
Epoch 15 - Train Loss: